In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
from NN_model import DNNClassifier, CNNClassifier, GridSearchCNN, GridSearchDNN

In [1]:
processedDataX_path = "preprocessedSamples_X_samples_allGuitar_20480_Mm7_R1B.data"
processedDatay_path = "preprocessedSamples_y_samples_allGuitar_20480_Mm7_R1B.data"
processedData_path = ""

if os.path.isfile(processedDataX_path): #if already preprocessed
	processedX = np.load(processedDataX_path)
	processedy = np.load(processedDatay_path)
else:
	import pandas as pd
	import librosa
	df = pd.read_csv('samples_allGuitar_20480_Mm7_R1B.csv')
	# df = pd.read_csv('samples_nylonGuitar_1024_Mm7_R03.csv')
	# df = pd.read_csv('../CachedData.csv')

	X_load = np.array(df.iloc[:,:-1], dtype=np.float)
	y_load = np.array(df.iloc[:,-1], dtype=np.float)
	processedX = np.zeros((len(X_load),12,80,1), dtype=np.float)
	processedy = np.zeros(len(y_load), dtype=np.float)
	for i in range(len(X_load)):
		# sample = librosa.core.stft(y=X_load[i], n_fft=2048, win_length=128, window='hamming', center=True, dtype=np.float32, pad_mode='reflect')
		sample = librosa.feature.chroma_stft(y=X_load[i], sr=44100, n_fft=20480, hop_length=258)
		sample = np.atleast_3d(sample)
		processedX[i] = sample
		processedy[i] = y_load[i]
		if i % 400 == 0:
			print(i)
	
	processedX.dump(processedDataX_path)
	processedy.dump(processedDatay_path)

from sklearn.utils import shuffle
print(processedy)
sprocessedX, sprocessedy = shuffle(processedX, processedy)
print(len(sprocessedX))
	
trainRange = int(len(sprocessedX) * 0.9)
validRange = int(len(sprocessedX) * 1)
testRange = int(len(sprocessedX) * 0.1)


X_train = np.array(sprocessedX[:trainRange], dtype=np.float)
y_train = np.array(sprocessedy[:trainRange], dtype=np.float)

X_valid = np.array(sprocessedX[trainRange:validRange], dtype=np.float)
y_valid = np.array(sprocessedy[trainRange:validRange], dtype=np.float)

X_test = np.array(sprocessedX[testRange:], dtype=np.float)
y_test = np.array(sprocessedy[testRange:], dtype=np.float)
print(y_test[1])
print(sprocessedX.shape)

NameError: name 'os' is not defined

In [22]:
import tensorflow as tf

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu

params = {'n_hidden_layers': [5],
	'n_neurons' : [1000],
	'optimizer_class' : [tf.train.AdagradOptimizer],
	'learning_rate' : [0.05],
	'batch_size' : [50],
	'activation' : [leaky_relu()],
	'dropout_rate' : [0.4],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.relu}],
	'conv2' : [{'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.3, 'conv2_activation':tf.nn.relu}],
    'architecture' : [2]
}

grid_search = GridSearchCNN(params, k_fold=5)
grid_search.fit(X_train, y_train, X_valid, y_valid)

Trining CNN with parameters: n_hidden_layers: 5, n_neurons: 1000, optimizer_class: <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, learning_rate: 0, batch_size: 50, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000014D2086ABF8>, dropout_rate: 0, architecture: 2, conv1: {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.1, 'conv1_activation': <function relu at 0x0000014D1DB1A048>}, conv2: {'conv2_fmaps': 32, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x0000014D1DB1A048>} .
Training and testing fold 0
ranges:
1684 2105
shapes:
(1687, 12, 80, 1) (421, 12, 80, 1)
(1687,) (421,)
Epoch 0, train accuracy: 30.0000%, valid. accuracy: 28.0851%, valid. best loss: 3.136065
Epoch 1, train accuracy: 55.0000%, valid. accuracy: 39.5745%, valid. best loss: 2.048710
Epoch 2, train accuracy: 45.0000%, valid. accuracy: 52.3404%, valid. best loss: 1.740090
Epoch 3, train accuracy: 70.0000

KeyboardInterrupt: 

In [ ]:
# global cnn
cnn = CNNClassifier(batch_size=50, learning_rate=0.01, n_hidden_layers=5, n_neurons=700, 
                    optimizer_class=tf.train.AdagradOptimizer, activation=leaky_relu(alpha=0.01), 
                    conv1={'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2},
                   conv2={'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.4})
range1 = 0
range2 = 350
X_train2 = X_train[:range1]
X_train2 = np.vstack([X_train2, X_train[range2:]])
print(X_train2.shape)
y_train2 = y_train[:range1]
y_train2 = np.append(y_train2, y_train[range2:])

X_test2 = X_test[range1:range2]
y_test2 = y_test[range1:range2]

#cnn.fit(X_train=X_train[350:], y_train=y_train[350:], X_valid=X_valid, y_valid=y_valid)
cnn.fit(X_train=X_train2, y_train=y_train2, X_valid=None, y_valid=None)

score = cnn.accuracy_score(X_test2, y_test2)
score

In [15]:
import tensorflow as tf

X_train_reshaped = np.reshape(X_train,(len(X_train), -1))
X_valid_reshaped = np.reshape(X_valid,(len(X_valid), -1))
X_test_reshaped = np.reshape(X_test,(len(X_test), -1))

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu

params = {'n_hidden_layers': [5],
	'n_neurons' : [700],
	'optimizer_class' : [tf.train.AdagradOptimizer],
	'learning_rate' : [0.05],
	'batch_size' : [50],
	'activation' : [leaky_relu()],
	'dropout_rate' : [0.1]
}

grid_search = GridSearchDNN(params, k_fold=5)
grid_search.fit(X_train_reshaped, y_train, X_valid_reshaped, y_valid)

Trining DNN with parameters: n_hidden_layers: 5, n_neurons: 700, optimizer_class: <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, learning_rate: 0, batch_size: 50, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000014D21404598>, dropout_rate: 0, 
Training and testing fold 0
ranges:
1684 2105
shapes:
(1687, 960) (421, 960)
(1687,) (421,)
0	Validation loss: 1.927824	Best loss: 1.927824	Accuracy: 41.28%
1	Validation loss: 1.332642	Best loss: 1.332642	Accuracy: 53.62%
2	Validation loss: 1.081540	Best loss: 1.081540	Accuracy: 67.23%
3	Validation loss: 1.061828	Best loss: 1.061828	Accuracy: 67.23%
4	Validation loss: 1.186352	Best loss: 1.061828	Accuracy: 63.83%
5	Validation loss: 0.728708	Best loss: 0.728708	Accuracy: 76.60%
6	Validation loss: 1.158489	Best loss: 0.728708	Accuracy: 72.77%
7	Validation loss: 0.832701	Best loss: 0.728708	Accuracy: 75.32%
8	Validation loss: 0.838882	Best loss: 0.728708	Accuracy: 75.74%
9	Validation loss: 0.788303	Best los

0	Validation loss: 2.012184	Best loss: 2.012184	Accuracy: 40.00%
1	Validation loss: 1.399388	Best loss: 1.399388	Accuracy: 59.15%
2	Validation loss: 0.922173	Best loss: 0.922173	Accuracy: 68.09%
3	Validation loss: 0.912935	Best loss: 0.912935	Accuracy: 71.49%
4	Validation loss: 0.908723	Best loss: 0.908723	Accuracy: 71.49%
5	Validation loss: 0.802367	Best loss: 0.802367	Accuracy: 74.47%
6	Validation loss: 0.947047	Best loss: 0.802367	Accuracy: 68.94%
7	Validation loss: 0.665976	Best loss: 0.665976	Accuracy: 78.72%
8	Validation loss: 0.714143	Best loss: 0.665976	Accuracy: 76.60%
9	Validation loss: 0.967501	Best loss: 0.665976	Accuracy: 72.34%
10	Validation loss: 0.673516	Best loss: 0.665976	Accuracy: 80.85%
11	Validation loss: 0.860106	Best loss: 0.665976	Accuracy: 78.30%
12	Validation loss: 0.701793	Best loss: 0.665976	Accuracy: 81.70%
13	Validation loss: 0.784266	Best loss: 0.665976	Accuracy: 81.28%
14	Validation loss: 0.699730	Best loss: 0.665976	Accuracy: 82.98%
15	Validation loss: 

45	Validation loss: 0.884266	Best loss: 0.674721	Accuracy: 85.11%
46	Validation loss: 0.931647	Best loss: 0.674721	Accuracy: 85.11%
47	Validation loss: 0.914701	Best loss: 0.674721	Accuracy: 85.11%
48	Validation loss: 0.911927	Best loss: 0.674721	Accuracy: 84.68%
49	Validation loss: 0.930058	Best loss: 0.674721	Accuracy: 84.26%
50	Validation loss: 0.931550	Best loss: 0.674721	Accuracy: 85.11%
51	Validation loss: 0.955821	Best loss: 0.674721	Accuracy: 85.53%
Early stopping!
Final accuracy on test set: 0.793349
Trining DNN with parameters: n_hidden_layers: 5, n_neurons: 700, optimizer_class: <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, learning_rate: 0, batch_size: 50, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000014D21404598>, dropout_rate: 0, 
Training and testing fold 4
ranges:
0 421
shapes:
(1687, 960) (421, 960)
(1687,) (421,)
0	Validation loss: 2.041788	Best loss: 2.041788	Accuracy: 35.74%
1	Validation loss: 1.408095	Best loss: 1.4080

Trining DNN with parameters: n_hidden_layers: 5, n_neurons: 1000, optimizer_class: <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, learning_rate: 0, batch_size: 50, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001F1E24536A8>, dropout_rate: 0, 
Training and testing fold 0
ranges:
1540 1925
shapes:
(1544, 960) (385, 960)
(1544,) (385,)


TypeError: fit() got an unexpected keyword argument 'X_train'